In [8]:
#------------------------------HEADER------------------------------#
#Projeto computacional em que implementamos o método simplex.
'''AUTHORS:
Daniel Reis, Matheus Alves, Mariane Santana.
'''

import numpy as np
#------------------------------HEADER------------------------------#

In [16]:

#Função que lê da entrada as informações relevantes para o PL.
def entrada():
    #Lê o tamanho da matriz de restrições
    linhas = int(input("Informe o tamanho da matriz.\nLinhas: "))
    colunas = int(input("\nColunas: "))
    #Cria uma matriz (A)mxn de zeros
    A = np.zeros(linhas*colunas, dtype=float).reshape(linhas, colunas)

    #Atribui às entradas de A os valores do input do usuário.
    for i in range(linhas):
        A[i] = input().split()

    #Atribui às entradas de b (vetor de recursos) os valores do input do usuário.
    recursos = [float(b) for b in input("Informe o vetor dos recursos ("+str(linhas)+" entradas separadas por um espaço): ").split(sep = " ")]

    #Atribui às entradas de c (vetor de custos) os valores do input do usuário.
    custos = [float(c) for c in input("Informe o vetor dos custos ("+str(colunas)+" entradas separadas por um espaço): ").split(sep = " ")]

    #Printa os valores lidos para confirmar se tudo correu bem.
    print('A = {}\n'.format(A))
    print('b = {}\n'.format(recursos))
    print('c = {}\n'.format(custos))
    return A, recursos, custos, linhas, colunas


In [17]:
A, recursos, custos, linhas, colunas = entrada()

A = [[3. 4. 5.]]

b = [4.0]

c = [5.0, 5.0, 6.0]



In [ ]:

#Função que particiona o PL.
def part(A,custos, linhas, colunas):
    # Define partições e os custos básicos e não-básicos
    particao_basica = A[:,(linhas-colunas):].copy()
    particao_nbasica = A[:,:linhas].copy()
    custos_basicos = custos[(linhas-colunas):].copy()
    custos_nbasicos = custos[:linhas].copy()
    return particao_basica, particao_nbasica, custos_basicos, custos_nbasicos

particao_basica, particao_nbasica, custos_basicos, custos_nbasicos = part(A,custos, linhas, colunas)

Loop daqui pra baixo

In [ ]:
# Procura uma solução básica factível, Xb.
def solucao_basica(particao_basica, recursos):
    xb = np.linalg.solve(particao_basica, recursos)
    if(min(xb) < 0 or min(xb) == None):
        print("Solucao basica nao encontrada.")
        return None
    else:
        return xb
#A funcao acima (solucao_basica) deverá ser chamada em loop até testar todas as particoes possiveis ou encontrar uma solucao factivel?

# def busca_sol_basica(A,recursos):
#     # Define partições e os custos básicos e não-básicos
#     particao_basica = A[:,(linhas-colunas):].copy()
#     particao_nbasica = A[:,:linhas].copy()
#     custos_basicos = custos[(linhas-colunas):].copy()
#     custos_nbasicos = custos[:linhas].copy()
#     #Iterar
#     solucao_basica(particao_basica,recursos)

xb = np.linalg.solve(particao_basica, recursos)
print(xb)


[10.  7. 60.]


In [ ]:
#Função que calcula o valor da função objetivo considerando apenas a parte básica.
def objetivo(custos_basicos, xb):
    valor_atual = np.dot(np.transpose(custos_basicos), xb)
    return valor_atual

valor_atual = objetivo(custos_basicos, xb)

-120.0


In [ ]:
#Função que calcula os custos relativos (e retorna o quê exatamente? Só os custos?) Ver tbm se a entrada recebe os parametros certos
def relativos(particao_nbasica, custos_nbasicos):
    return 0

# Calcular os custos relativos
lambda_simplex = np.linalg.solve(np.transpose(particao_basica), custos_basicos)
custos_relativos = [(custos_nbasicos[j] - np.dot(np.transpose(lambda_simplex), particao_nbasica[:,j])) for j in range(colunas-linhas)]
custo_relativo_min = min(custos_relativos)
indice_entrada_base = custos_relativos.index(custo_relativo_min)

In [ ]:
print(lambda_simplex)
print(custos_relativos)
print(custo_relativo_min)
print(indice_entrada_base)

[-1.2  0.   0. ]
[1.2, -0.6000000000000001, 0.19999999999999996]
-0.6000000000000001
1


In [ ]:
#Funcao que performa o teste de otimalidade (Essa é tão simples que fico na duvida se cabe implementar direto no main()).
def otima(custos_relativos):
    custo_relativo_min = min(custos_relativos)
    if(custo_relativo_min >= 0):
        return True
    else:
        return False
    


# Teste de otimalidade
if (custo_relativo_min >= 0):
    print("Solução atual é ótima.")
    print("Valor ótimo da solução: ", valor_atual)
    print("Variáveis ótimas: ", xb)
else:
    print("Solução atual não é ótima.")
    print("Valor atual da solução: ", valor_atual)
    print("Variáveis: ", xb)

Solução atual não é ótima.
Valor atual da solução:  -120.0
Variáveis:  [10.  7. 60.]


In [ ]:
# Cálculo da Direção Simplex
direcao_simplex = np.linalg.solve(particao_basica, particao_nbasica[:,indice_entrada_base])
print(direcao_simplex)

[0.2 1.6 3.6]


In [ ]:

if True in (direcao_simplex <= 0):
    print("Problema não tem solução ótima finita.")
else:
    passo = xb/direcao_simplex
    epsilon = min(passo)
    indice_saida_base = np.where(passo == epsilon)[0][0]

In [ ]:
print(passo)
print(epsilon)
print(indice_saida_base)

[50.          4.375      16.66666667]
4.375
1


In [ ]:
entrada_base = particao_nbasica[:,indice_entrada_base].copy()
saida_base = particao_basica[:,indice_saida_base].copy()
particao_basica[:,indice_saida_base] = entrada_base
particao_nbasica[:,indice_entrada_base] = saida_base

In [ ]:
entrada_custo = custos_nbasicos[indice_entrada_base]
saida_custo = custos_basicos[indice_saida_base]
custos_basicos[indice_saida_base] = entrada_custo
custos_nbasicos[indice_entrada_base] = saida_custo

In [ ]:
print(particao_basica)
print(particao_nbasica)

[[10.  0.  0.]
 [ 7.  1.  0.]
 [ 2.  0.  1.]]
[[1. 2. 1.]
 [0. 3. 2.]
 [0. 4. 1.]]


In [ ]:
print(custos_basicos)
print(custos_nbasicos)

[-12.0, 0.0, 0.0]
[0.0, -3.0, -1.0]


In [ ]:
def main():
    
    A, recursos, custos, linhas, colunas = entrada()
    particao_basica, particao_nbasica, custos_basicos, custos_nbasicos = part(A,custos, linhas, colunas)
    xb = solucao_basica(particao_basica, recursos) #Verificar se precisamos iterar sobre todas as possiveis formas de particionar.
    if(xb == None):
        exit(0) #Termina a execução do programa.

    custos_relativos = relativos(particao_nbasica, custos_nbasicos)
    while(otima(custos_relativos) != True):
        custo_relativo_min = min(custos_relativos)
        indice_entrada_base = custos_relativos.index(custo_relativo_min)
        #direcao de passo
            #Se direcao é negativa ... O problema é ilimitado.
            #break;
        #indice de saida da base

        #Funcao que faz a troca (lembrar de informar quais variaveis estao em cada particao na iteracao atual).
        #refaz custo relativo




    if(otima(custos_relativos)):
        valor_atual = objetivo(custos_basicos, xb)

        print("Solução atual é ótima.")
        print("Valor ótimo da solução: ", valor_atual)
        print("Variáveis ótimas: ", xb)
        exit(0)


    
    